# Run the import block only

In [2]:
from dowhy import CausalModel
import pandas as pd
import numpy as np
import dowhy
from tqdm import tqdm

In [26]:
data = pd.read_csv('data/ml-100k/u.data', sep="\t", header=None)
data.columns = ['user_id', 'movie_id', 'rating', "time_stamp"]
data = data.drop('time_stamp', axis=1)

In [27]:
data["treatment"] = np.ones(data.shape[0], dtype=int)

In [28]:
data.shape

In [29]:
data.head()

,user_id,movie_id,rating,treatment
0,196,242,3,1
1,186,302,3,1
2,22,377,1,1
3,244,51,2,1
4,166,346,1,1


In [30]:
unique_users = data.user_id.unique().tolist()

In [31]:
unique_movie_ids = set(data.movie_id.unique().tolist())

In [32]:
grouped_df = data.groupby(by="user_id")

In [33]:
vertical_df = pd.DataFrame(columns=data.columns)

In [34]:
del data

In [35]:
for user in tqdm(unique_users):
    temp_1 = grouped_df.get_group(user)
    
    missing_movies = list(unique_movie_ids - set(temp_1.movie_id))
    count = len(missing_movies)
    
    temp_2 = {
        "user_id": [user] * count,
        "movie_id": missing_movies,
        "rating": [0] * count,
        "treatment": [0] * count
    }

    new_rec = pd.concat([temp_1, pd.DataFrame(temp_2)], ignore_index=True)
    vertical_df = pd.concat([vertical_df, new_rec], ignore_index=True)
    
    del new_rec

100%|██████████| 943/943 [00:33<00:00, 28.55it/s] 


In [36]:
vertical_df.shape

In [16]:
# run it only when you want to create a csv of preprocessed data for dowhy causal model

#vertical_df.to_csv("data/preprocess_data/preproc_100k_dowhy.csv", index=False)

In [37]:
# uncomment only when excuting the all the blocks of code 

#del vertical_df

# Execute the code from here

In [3]:
vertical_df = pd.read_csv("data/preprocess_data/preproc_100k_dowhy.csv")

In [4]:
vertical_df.treatment = vertical_df.treatment.astype('bool')
vertical_df.user_id = vertical_df.user_id.astype(np.uint16)
vertical_df.movie_id = vertical_df.movie_id.astype(np.uint16)
vertical_df.rating = vertical_df.rating.astype(np.uint8)

In [5]:
vertical_df.head()

,user_id,movie_id,rating,treatment
0,196,242,3,True
1,196,393,4,True
2,196,381,4,True
3,196,251,3,True
4,196,655,5,True


In [6]:
train = vertical_df.sample(frac = 0.8) 

In [7]:
test = vertical_df.drop(train.index)

In [8]:
model=CausalModel(
        data = train,
        treatment='treatment',
        outcome='rating',
        common_causes = ["user_id", "movie_id"]
        )

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['rating']


In [9]:
identified_estimand = model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['movie_id', 'user_id', 'U']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] n


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                            
────────────(Expectation(rating|movie_id,user_id))
d[treatment]                                      
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→rating then P(rating|treatment,movie_id,user_id,U) = P(rating|treatment,movie_id,user_id)
### Estimand : 2
Estimand name: iv
No such variable found!



In [10]:
estimate = model.estimate_effect(identified_estimand, 
                                 method_name="backdoor.linear_regression", 
                                 test_significance=True)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: rating~treatment+movie_id+user_id


                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                 3.889e+06
Date:                Fri, 13 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:35:53   Log-Likelihood:            -1.9755e+05
No. Observations:             1268901   AIC:                         3.951e+05
Df Residuals:                 1268897   BIC:                         3.951e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0185      0.001     26.943      0.0

In [11]:
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                            
────────────(Expectation(rating|movie_id,user_id))
d[treatment]                                      
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→rating then P(rating|treatment,movie_id,user_id,U) = P(rating|treatment,movie_id,user_id)
### Estimand : 2
Estimand name: iv
No such variable found!

## Realized estimand
b: rating~treatment+movie_id+user_id
Target units: ate

## Estimate
Mean value: 3.51983478583828
p-value: [0.]



In [12]:
print("Causal Estimate is " + str(estimate.value))

Causal Estimate is 3.51983478583828


In [13]:
refute_results=model.refute_estimate(identified_estimand, estimate,
        method_name="random_common_cause")

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: rating~treatment+movie_id+user_id+w_random


                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                 2.917e+06
Date:                Fri, 13 Nov 2020   Prob (F-statistic):               0.00
Time:                        22:36:01   Log-Likelihood:            -1.9754e+05
No. Observations:             1268901   AIC:                         3.951e+05
Df Residuals:                 1268896   BIC:                         3.952e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0185      0.001     26.944      0.0

In [14]:
print(refute_results)

Refute: Add a Random Common Cause
Estimated effect:3.51983478583828
New effect:3.5198339845080993

